<a href="https://colab.research.google.com/github/AVmaster84/Web-Development/blob/main/crapy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare env, path , file

In [ ]:
# @title **worksheet**
# @markdown Forms support many types of fields.
import datetime
victims = "hacom" # @param ["hacom","phongvu","thegioididong","vuhoang","cellphones"] {"allow-input":true}
victim_url = "https://hacom.vn" # @param ["https://hacom.vn/","https://thegioididong.com/","https://www.phongvu.com/","https://cellphones.com.vn/"] {"allow-input":true}
# slider_value = 155  # @param {type: "slider", min: 100, max: 200}
# number = 102  # @param {type: "number"}
date = datetime.date.today().strftime('%d-%b-%Y')
vic_category = "laptop" # @param {"type":"string","placeholder":"Danh mục"}
category_id = "" # @param {"type":"string","placeholder":"ID danh mục"}
total_pages = "" # @param {"type":"string","placeholder":"Tong so trang"}
other_param = "" # @param {"type":"string","placeholder":"thông số khác"}
# @markdown ---

In [ ]:
#@title setup cloudinary api
!pip install cloudinary selenium beautifulsoup4 requests pandas urllib3
from IPython.display import clear_output
clear_output()
from google.colab import drive
import cloudinary
import cloudinary.api
cloudinary.config(
  cloud_name = 'dzdfldd9j',
  api_key = userdata.get('cloudinarydnk_key'),
  api_secret = userdata.get('cloudinarydnk_secret'),
  secure = True
)

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/tmp')

# Function

In [ ]:
#@title Trích xuất link từ trang danh mục

from posix import readlink
import requests
from bs4 import BeautifulSoup
import re
import os
import time
from IPython.display import clear_output

def extract_links(url, output_file):
    try:
        # Gửi request tới trang
        response = requests.get(url)
        time.sleep(1)
        response.raise_for_status()

        # Phân tích HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # # Tìm tất cả phần tử sản phẩm
        product_items = soup.find_all('h3')

        # Lọc và lưu links
        filtered_links = []
        base_url = base_url
        for item in product_items:
            # Tìm thẻ a trong mỗi item
            link = item.find('a', href=True)
            if link:
            # full_link = requests.compat.urljoin(base_url, link['href'])
              full_link = requests.compat.urljoin(url, link['href'])
              filtered_links.append(full_link)

        # Ghi links ra file
        with open(output_file, 'w', encoding='utf-8') as f:
            for link in filtered_links:
                f.write(f"{link}\n")
        with open(output_file, 'w', encoding='utf-8') as f:
            for link in filtered_links:
                f.write(f"{link}\n")

        print(f"Đã tìm thấy và lưu {len(filtered_links)} links vào {output_file}")
        return filtered_links

    except requests.RequestException as e:
        print(f"Lỗi khi truy xuất trang: {e}")
        return []

# Chuỗi main
def main():
    # URL danh mục
    category_name = vic_category
    url = f'{base_url}-{category_name}'

    # File lưu links
    output_file =os.path.join(working_path, f'{category_name}+{date}.txt')

    # Bước 1: Trích xuất links
    extract_links(url, output_file)

# Chạy script
if __name__ == "__main__":
    main()

#  Trích xuất thông tin: tên sp, giá, mô tả ngắn

# OK

In [ ]:
#@title Get tên, thông số, giá, hình ảnh chính từ trang category
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import json
import time
import pandas as pd
from datetime import datetime
from IPython.display import clear_output

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Chạy ẩn browser
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=chrome_options)
    return driver

def get_products_by_page(driver, page, cat_id="144", collection_id=""):
    try:
        # Tạo URL AJAX
        ajax_url = f"/ajax/get_json.php?action=product&action_type=product-list&category={cat_id}&collection={collection_id}&show=36&page={page}&sort=order-last-update"

        # Thực thi AJAX call
        response = driver.execute_script(f"""
            return fetch('{ajax_url}')
                .then(response => response.json())
                .catch(error => console.error('Error:', error));
        """)

        if not response:
            print(f"Warning: No response for page {page}")
            return None

        return response
    except Exception as e:
        print(f"Error fetching page {page}: {str(e)}")
        return None

def process_product_data(product):
    try:
        product_image = product.get('productImage', {}).get('large')
        original_image = product_image.replace("250_", '')
        file_name = original_image.rsplit("/", 1)[-1]
        cloudinary.uploader.upload(original_image,
                                    asset_folder = "laptop",
                                    use_filename = True,
                                    resource_type = "image",
                                    type = "upload",
                                    unique_filename = False)
        img_url = CloudinaryImage(file_name).build_url()
        print(f"Đã tải upload {file_name} ")
        time.sleep(1)
        return {
            'id': product.get('id'),
            'name': product.get('productName'),
            'price': product.get('price'),
            'link': product.get('productUrl'),
            # 'image': product.get('productImage', {}).get('large'),
            'image': img_url,
            # 'collection': images_collection,
            'summary': product.get('productSummary'),
            'brand': product.get('brand', {}).get('name'),
        }

    except Exception as e:
        print(f"Error processing product: {str(e)}")
        product_image = product.get('productImage', {}).get('large')
        file_name = product_image.rsplit("/", 1)[-1]
        cloudinary.uploader.upload(product_image,
                                    asset_folder = "laptop",
                                    use_filename = True,
                                    resource_type = "image",
                                    type = "upload",
                                    unique_filename = False)
        img_url = CloudinaryImage(file_name).build_url()
        print(f"Đã tải upload {file_name} ")
        time.sleep(1)
        return {
            'id': product.get('id'),
            'name': product.get('productName'),
            'price': product.get('price'),
            'link': product.get('productUrl'),
            # 'image': product.get('productImage', {}).get('large'),
            'image': img_url,
            # 'collection': images_collection,
            'summary': product.get('productSummary'),
            'brand': product.get('brand', {}).get('name'),
        }


def scrape_all_products(base_url, cat_id="144", total_pages=17):
    driver = setup_driver()
    all_products = []

    try:
        # Truy cập trang web để initial session
        driver.get(base_url)
        time.sleep(2)  # Đợi trang load

        for page in range(1, total_pages + 1):
            print(f"Scraping page {page}/{total_pages}...")

            data = get_products_by_page(driver, page, cat_id)
            if not data:
                continue

            # Xử lý từng sản phẩm
            for product in data['list']:
                processed_product = process_product_data(product)
                all_products.append(processed_product)
                # processed_image = scrape_all_image(product)
                # if processed_image:
                #   all_products.append(processed_image)

            print(f"Got {len(data['list'])} products from page {page}")
            time.sleep(1)  # Delay nhẹ giữa các request

    except Exception as e:
        print(f"Error during scraping: {str(e)}")

    finally:
        driver.quit()

    return all_products

def save_to_csv(products, filename=None):
    if not filename:
        filename = f"products_{datetime.now().strftime('%d%m%Y_%H%M')}.csv"

    df = pd.DataFrame(products)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    time.sleep(1)
    clear_output()
    print(f"Saved {len(products)} products to {filename}")

# Sử dụng
if __name__ == "__main__":
    #@markdown Thay đổi URL theo nhu cầu
    BASE_URL = "https://hacom.vn/chuot-may-tinh"  #@param {type:"string"}
    #@markdown Category ID của laptop Acer
    CATEGORY_ID = "38" #@param {type:"string"}
    #@markdown Tổng số trang
    TOTAL_PAGES = 13  #@param {type:"integer" }
    filename = os.path.join('/content/drive/MyDrive/tmp', f"{BASE_URL.split('/')[-1]}_{datetime.now().strftime('%d%m%Y_%H%M')}.csv")
    products = scrape_all_products(BASE_URL, CATEGORY_ID, TOTAL_PAGES)
    save_to_csv(products,filename=filename)

Saved 446 products to /content/drive/MyDrive/tmp/chuot-may-tinh_26122024_0909.csv


# Mục mới

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import cloudinary
import cloudinary.api
import cloudinary.uploader
from cloudinary import CloudinaryImage
import time

df = pd.read_csv('thiet-bi-chia-mang-switch_26122024_0416.csv')
df['image_collection'] = ''

for index, row in df.iterrows():
    # images_collection = []
    product_url = row['link']
    product_name = row['name']
    full_url = "https://hacom.vn"+product_url
    requests_response = requests.get(full_url)
    requests_response.raise_for_status()
    soup_product = BeautifulSoup(requests_response.text,'html.parser')
    div_image = soup_product.find('div', class_="pd-info-left")
  # a_href = div_image.find_all('a', class_="owl-thumb-item")
    images = soup_product.find_all('img')
    images_collection = []
    for image in images:
      if image and image.find_parent('a', class_='owl-thumb-item item js-pd-image-item'):
        img_src = image.get('src')
        image_filename = img_src.rsplit("/", 1)[-1]
        if image_filename.startswith('120_'):
          image_filename = image_filename.replace("120_", '')
          img_fullsize = img_src.replace(img_src.rsplit("/", 1)[-1], image_filename)
          print(img_fullsize)

          cloudinary.uploader.upload(img_fullsize,
                            asset_folder = "laptop",
                              use_filename = True,
                              resource_type = "image",
                              type = "upload",
                              unique_filename = False)
          image_uploaded = CloudinaryImage(image_filename).build_url()
          images_collection.append(image_uploaded)
          time.sleep(1)

    print(images_collection)
    print(f"đã tìm thấy {len(images_collection)} của {product_name}")
    df.at[index,'image_collection'] = images_collection

df.to_csv('thiet-bi-chia-mang-switch_26122024_0416_with_collection.csv', index=False)


[]
đã tìm thấy 0 của Switch Cisco CBS110-24T-EU Unmanaged 24-port GE, 2x1G SFP Shared
[]
đã tìm thấy 0 của Switch TP-Link TL-SF1048 (48 Port 10/100Mbps - Vỏ kim loại)
https://hanoicomputercdn.com/media/product/5894_tl_sg1024d.jpg
https://hanoicomputercdn.com/media/product/5894_tl_sg1024d_3.jpg
https://hanoicomputercdn.com/media/product/5894_tl_sg1024d_2.jpg
https://hanoicomputercdn.com/media/product/5894_tl_sg1024d_1.jpg
https://hanoicomputercdn.com/media/product/5894_z2007966218850_d797209f29fee72607cda41d87da452b.jpg
https://hanoicomputercdn.com/media/product/5894_z2007966218849_af69ccc327888fe048e79a95b599a645.jpg
https://hanoicomputercdn.com/media/product/5894_z2007966211951_35e4771e0b97dee425ad20ab74d580c1.jpg
https://hanoicomputercdn.com/media/product/5894_tl_sg1024d_03.jpg
['https://res.cloudinary.com/dzdfldd9j/image/upload/5894_tl_sg1024d.jpg', 'https://res.cloudinary.com/dzdfldd9j/image/upload/5894_tl_sg1024d_3.jpg', 'https://res.cloudinary.com/dzdfldd9j/image/upload/5894_tl_s